### Import Libraries

In [ ]:
!pip install xlsxwriter
import xlsxwriter
import xlrd
import openpyxl
from openpyxl import load_workbook
import os
import os.path
from os import path
import datetime as dt
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import csv
import sqlite3 as sql
import pandas as pd
pd.options.mode.chained_assignment = None
from google.colab import drive
drive.mount('/content/drive')
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from nltk import sent_tokenize, word_tokenize
nltk.download('punkt')
nltk.download('wordnet')

     |████████████████████████████████| 149 kB 10.7 MB/s 
Mounted at /content/drive
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
display_images = True

In [ ]:
os.chdir("drive/My Drive") # after you run this once you will get an error if you run again. You can ignore

2018 Data: Between February and November     
713,534 articles  
4 features:  
date, source, name (headline), content (text).  
194 unique sources.   
3106 Fox News articles.  
8202 CNN articles.   
2426 CNBC articles.    
6604 MSNBC articles.  




In [ ]:
cnx_2018 = sql.connect('USML_Project/Data/2018_articles.db')
# cnx_2019 = sql.connect('USML_Project/Data/2019_articles.db')
# cnx_2020 = sql.connect('USML_Project/Data/2020_articles.db')

## Extract Specific Sources

In [ ]:
# nbc_2018 = pd.read_sql_query("SELECT * FROM articles WHERE source LIKE '%NBC%'", cnx_2018)
# nbc_2018['date']= pd.to_datetime(nbc_2018['date'])

fox_2018 = pd.read_sql_query("SELECT * FROM articles WHERE source LIKE '%Fox News%'", cnx_2018)
# fox_2018['date']= pd.to_datetime(fox_2018['date'])

cnn_2018 = pd.read_sql_query("SELECT * FROM articles WHERE source LIKE '%CNN%'", cnx_2018)
# cnn_2018['date']= pd.to_datetime(cnn_2018['date'])

#Rest of the years
# fox_2019 = pd.read_sql_query("SELECT * FROM articles WHERE source LIKE '%Fox News%'", cnx_2019)
# fox_2020 = pd.read_sql_query("SELECT * FROM articles WHERE source LIKE '%Fox News%'", cnx_2020)
# cnn_2019 = pd.read_sql_query("SELECT * FROM articles WHERE source LIKE '%CNN%'", cnx_2019)
# cnn_2020 = pd.read_sql_query("SELECT * FROM articles WHERE source LIKE '%CNN%'", cnx_2020)

In [ ]:
#Prepare the data: Topic Model, Train
#Uncomment the rest if you plan to train two separate topic models
#Also have to resetindex and other stuff

fox_2018_sampled = fox_2018.sample(n=3000, random_state=1)
# fox_2018_topic_model_train = fox_2018_sampled.sample(frac=.75, random_state=1) #0.75
# fox_2018_test = fox_2018_sampled.drop(fox_2018_topic_model_train.index)


cnn_2018_sampled = cnn_2018.sample(n=3000, random_state=1)
# cnn_2018_topic_model_train = cnn_2018_sampled.sample(frac=.75, random_state=1) #1000
# cnn_2018_test = cnn_2018_sampled.drop(cnn_2018_topic_model_train.index)

## Add length of headline and article features ->  LATER

In [ ]:
##MIGHT NEED WHEN TRAINING MODEL
# cnn_2018['article_length'] = cnn_2018['content'].str.len().values
# cnn_2018['headline_length'] = cnn_2018['name'].str.len().values

In [ ]:
from string import punctuation
from nltk.stem.wordnet import WordNetLemmatizer
Cstopwords=set(stopwords.words('english')+list(punctuation)+['from', 'subject', 're', 'edu', 'use', 'https', 'said', 'us', 'would', 'like','says', 'could', 'two', 'told', 'going', 'one'])
lemma=WordNetLemmatizer()
def clean_article(dialogue_column):
    dialogue_corpus=[]
    for i in range(0,len(dialogue_column)):
        dialogue=dialogue_column[i]
        dialogue=str(dialogue).lower()
        dialogue=word_tokenize(dialogue)
        dialogue = [word.lower() for word in dialogue if word.isalpha() and word not in Cstopwords]
        dialogue=[lemma.lemmatize(w) for w in dialogue ]
        dialogue_corpus.append(dialogue)
    return dialogue_corpus

In [ ]:
combined_topic_model = pd.concat([fox_2018_sampled,cnn_2018_sampled])

In [ ]:
combined_topic_model = combined_topic_model.reset_index(drop=True)
#combined_test = combined_test.reset_index(drop=True)

In [ ]:
combined_topic_model.shape

(6000, 4)

In [ ]:
combined_topic_model["content"]=clean_article(combined_topic_model["content"])

In [ ]:
#fox_2018['content'] = [', '.join(map(str, l)) for l in fox_2018['content']]
##Convert the content into string from list format if needed

In [ ]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(combined_topic_model["content"])
# Create Corpus
texts = combined_topic_model["content"]
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 2), (1, 1), (2, 2), (3, 1), (4, 1), (5, 2), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


In [ ]:
from pprint import pprint
num_topics = 20
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       passes=5)
doc_lda = lda_model[corpus]

In [ ]:
train_vecs = []
for i in range(len(combined_topic_model)):
    top_topics = lda_model.get_document_topics(corpus[i], minimum_probability=0.0) #corpus = bow
    topic_vec = [top_topics[i][1] for i in range(20)]
    # topic_vec.extend([combined_topic_model.iloc[i].real_counts]) # counts of reviews for restaurant #these two are new features, can include later
    # topic_vec.extend([len(combined_topic_model.iloc[i].text)]) # length review
    train_vecs.append(topic_vec)

In [ ]:
for num in train_vecs[0]:
  print("{:.4f}".format(num))
# "{:.7f}".format(num for num in train_vecs[0])
# [round(num, 4) for num in train_vecs[0]]
# train_vecs[0]

0.0708
0.0001
0.1477
0.0001
0.0109
0.0092
0.0001
0.0001
0.0003
0.2929
0.0001
0.1578
0.1141
0.0930
0.0001
0.0582
0.0001
0.0001
0.0440
0.0001


In [ ]:
#Change df of test with 1 or 0 for classification task
combined_topic_model['label'] = combined_topic_model['source'].map({'Fox News' : 0, 'CNN' : 1})

In [ ]:
X = np.array(train_vecs)
y = np.array(combined_topic_model['label'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2)

Logistic Regression

In [ ]:
#Accuracy metric is more relevant to us since the classes are balanced
#https://medium.com/analytics-vidhya/accuracy-vs-f1-score-6258237beca2

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(class_weight= 'balanced', solver='lbfgs',fit_intercept=True, max_iter=200).fit(X_train, y_train)
#newton-cg

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [ ]:
#Logistic Regression
round(f1_score(y_test, y_pred),2)

0.65

In [ ]:
round(accuracy_score(y_test,y_pred), 2)

0.63

SGD Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(max_iter=1000,tol=1e-3,loss='log',class_weight='balanced').fit(X_train, y_train)

In [ ]:
y_pred = sgd.predict(X_test)

In [ ]:
#SGDClassifier
round(f1_score(y_test, y_pred),2)

0.65

In [ ]:
round(accuracy_score(y_test,y_pred), 2)

0.63

XGBoost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

XGBClassifier()

In [ ]:
y_pred = xgb.predict(X_test)

In [ ]:
#XGBoost
round(f1_score(y_test, y_pred),2)

0.72

In [ ]:
round(accuracy_score(y_test,y_pred), 2)

0.71